<a href="https://colab.research.google.com/github/rushi-k12/API/blob/Gov_data/API_gov_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [8]:
import csv
import requests

def fetch_data_and_write_to_csv():
    url = "https://api.data.gov.in/resource/7b624b4a-1456-4945-80d0-dfb5e40ddcff"
    api_key = "579b464db66ec23bdd000001ad8162b89c3d421d5c671483bd3020de"
    format = "json"
    offset = 0
    limit = 1000

    response = requests.get(url, params={"api-key": api_key, "format": format, "offset": offset, "limit": limit})

    if response.status_code == 200:
        data = response.json()

        records = data.get("records", [])

        headers = records[0].keys() if records else []

        csv_file_path = "monthly_consumption_petroleum_products.csv"

        with open(csv_file_path, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=headers)

            writer.writeheader()

            for record in records:
                writer.writerow(record)

        print("CSV file has been created successfully.")
    else:
        print("Failed to fetch data. Status code:", response.status_code)

fetch_data_and_write_to_csv()


CSV file has been created successfully.


In [9]:
df = pd.read_csv("/content/monthly_consumption_petroleum_products.csv")
df

,_month_,year,products,quantity_000_metric_tonnes_,updated_date
0,July,2023,ATF,663.23,2024-01-03
1,February,2023,ATF,626.08,2024-01-03
2,January,2023,ATF,667.84,2024-01-03
3,June,2022,ATF,588.17,2024-01-03
4,January,2022,ATF,455.52,2024-01-03
...,...,...,...,...,...
487,July,2021,Bitumen,434.65,2024-01-03
488,April,2021,Bitumen,818.11,2024-01-03
489,October,2020,Bitumen,640.00,2024-01-03
490,May,2020,Bitumen,590.00,2024-01-03


In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("monthly_consumption_petroleum_products.csv")

X = data[['year', 'quantity_000_metric_tonnes_']]
y = data['products']

y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


Epoch 1/50
10/10 - 1s - loss: 2.4872 - accuracy: 0.0287 - val_loss: 2.4584 - val_accuracy: 0.0886 - 1s/epoch - 140ms/step
Epoch 2/50
10/10 - 0s - loss: 2.4457 - accuracy: 0.1306 - val_loss: 2.4337 - val_accuracy: 0.1392 - 93ms/epoch - 9ms/step
Epoch 3/50
10/10 - 0s - loss: 2.4134 - accuracy: 0.2102 - val_loss: 2.4116 - val_accuracy: 0.1392 - 68ms/epoch - 7ms/step
Epoch 4/50
10/10 - 0s - loss: 2.3782 - accuracy: 0.1943 - val_loss: 2.3858 - val_accuracy: 0.1646 - 85ms/epoch - 8ms/step
Epoch 5/50
10/10 - 0s - loss: 2.3430 - accuracy: 0.2166 - val_loss: 2.3537 - val_accuracy: 0.1899 - 92ms/epoch - 9ms/step
Epoch 6/50
10/10 - 0s - loss: 2.3043 - accuracy: 0.2038 - val_loss: 2.3192 - val_accuracy: 0.1519 - 79ms/epoch - 8ms/step
Epoch 7/50
10/10 - 0s - loss: 2.2584 - accuracy: 0.1975 - val_loss: 2.2782 - val_accuracy: 0.1519 - 63ms/epoch - 6ms/step
Epoch 8/50
10/10 - 0s - loss: 2.2128 - accuracy: 0.1975 - val_loss: 2.2388 - val_accuracy: 0.1519 - 69ms/epoch - 7ms/step
Epoch 9/50
10/10 - 0s - 

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

data = pd.read_csv("monthly_consumption_petroleum_products.csv")

X = data[['year', 'quantity_000_metric_tonnes_']]
y = data['products']

y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='hyperparameter_tuning',
    project_name='monthly_consumption_petroleum'
)

tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

best_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


Trial 5 Complete [00h 00m 47s]
val_accuracy: 0.603375514348348

Best val_accuracy So Far: 0.6075949470202128
Total elapsed time: 00h 03m 07s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 160)               480       
                                                                 
 dense_1 (Dense)             (None, 160)               25760     
                                                                 
 dense_2 (Dense)             (None, 12)                1932      
                                                                 
Total params: 28172 (110.05 KB)
Trainable params: 28172 (110.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
10/10 [==============================] - 1s 31ms/step - loss: 0.8145 - accuracy: 0.6115 - val_loss: 0.9405 - val_accuracy: 0.5570


In [16]:
# Convert DataFrame to numpy array
y_test_np = y_test.to_numpy()

# Example: Confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = best_model.predict(X_test_scaled)
conf_matrix = confusion_matrix(y_test_np.argmax(axis=1), y_pred.argmax(axis=1))



4/4 [==============================] - 0s 4ms/step


In [18]:
# Example: Deploying a model using Flask
from flask import Flask, request, jsonify
app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prediction = model.predict(data)
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
